In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.signal import find_peaks

In [39]:
#Importing the CSV file
SYMBOL = 'eurusd'.upper()
path = f"../OHLC_data/{SYMBOL}_20240805_20240810.csv"

OHLC_df = pd.read_csv(path)

In [41]:
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'].astype('str') + ' ' + OHLC_df['time'].astype('str'), format='%d/%m/%Y %H:%M')
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'], format='%d/%m/%Y %H:%M')
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'])
OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'], dayfirst=True)
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'], format='%Y-%m-%d %H:%M')

In [43]:
OHLC_df.drop(["date","time"], axis=1)

,open,high,low,close,datetime
0,1.09221,1.09228,1.09221,1.09228,2024-05-08 00:00:00
1,1.09221,1.09228,1.09217,1.09228,2024-05-08 00:01:00
2,1.09232,1.09232,1.09185,1.09187,2024-05-08 00:02:00
3,1.09181,1.09196,1.09180,1.09196,2024-05-08 00:03:00
4,1.09198,1.09198,1.09198,1.09198,2024-05-08 00:04:00
...,...,...,...,...,...
6716,1.09133,1.09133,1.09124,1.09130,2024-09-08 15:56:00
6717,1.09130,1.09135,1.09125,1.09125,2024-09-08 15:57:00
6718,1.09125,1.09127,1.09119,1.09121,2024-09-08 15:58:00
6719,1.09121,1.09121,1.09113,1.09116,2024-09-08 15:59:00


In [45]:
#Functions
            #Pa2 = p1_range_localmaximas[condition5 & condition ].drop(['open','low','close'], axis=1)
            #Pa2 = Pa2.drop_duplicates(subset=['high'], keep='last')
            #Pa2 = Pa2.rename(columns={"high":"price"})
            #HH_HL_df = pd.concat([HH_HL_df, Pa2], ignore_index=True)

#def rename_column_price():
    # Pa = p1_range_localminimas[p1_range_localminimas['low'] == Pay].drop(['open','high','close'], axis=1)
    # Pa = Pa3.drop_duplicates(subset=['low'], keep='last')
    # Pa = Pa3.rename(columns={"low":"price"})
    

In [47]:
OHLC_df = OHLC_df.reindex(labels=["datetime","open","high","low","close"], axis=1)

In [49]:
#obtaining the OHLC data
start_dt="2024-09-08 9:00:00"
end_dt="2024-09-08 14:00:00"


rates_df = OHLC_df[(OHLC_df['datetime'] >= start_dt) & (OHLC_df['datetime'] <= end_dt)]

#OBTAINING THE HIGHEST POINT DURING THE NY SESSION
p3y = rates_df['high'].max()

#OBTAINING THE INDEX OF THE HIGHEST POINT
p3i = (rates_df[rates_df['high']== p3y])

#obtaining the OHLC data of P3
p3 = p3i.iloc[[-1]]

#converting the datatype of the datetime column
rates_df.loc[:,'datetime'] = pd.to_datetime(rates_df["datetime"])

#obtaining the data within the time limit 4 to 45 mins before P3
p1x_range_1 = p3["datetime"].iloc[0] - timedelta(minutes = 45)
p1x_range_2 = p3["datetime"].iloc[0] - timedelta(minutes = 0)
mask1= rates_df["datetime"].ge(p1x_range_1) & rates_df["datetime"].le(p1x_range_2)
p1_range = rates_df[mask1].reset_index(drop=True)

In [52]:
# Find the index of local maximas
local_maximas_i,_ = find_peaks(p1_range['high'].values)
local_maximas_i = np.array(local_maximas_i)

# Find the index of the local minima by inverting the data
local_minimas_i,_ = find_peaks(-p1_range['low'].values)
local_minimas_i = np.array(local_minimas_i)

#Obtaining the dataframe of the local minimas and local maximas
p1_range_localminimas = p1_range[p1_range.index.isin(local_minimas_i)].reset_index(drop=True)
p1_range_localmaximas = p1_range[p1_range.index.isin(local_maximas_i)].reset_index(drop=True)

In [54]:
#Finding the value of Pay1
Pay1 = p1_range_localminimas["low"].min()

#Finding the index of the first local minima
Pay1_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay1].index.min()


#Dropping the minimas before the first local minimas
mask = p1_range_localminimas.index >= Pay1_index
p1_range_localminimas = p1_range_localminimas[mask].reset_index(drop=True)
p1_range_localminimas
#Finding the index of the first local minima
Pay1_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay1].index.max()

In [56]:
Pay1_index

0

In [58]:
#to find the y value of the possible 2nd local minima:
Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
Pay3_index = compute_index_Pa_n(Pay3)
Pay3_index_range = Pay3_index

In [60]:
while True:
    try:
        #finding the index of the 1st local maxima
        condition1 = p1_range_localmaximas["datetime"].gt(p1_range_localminimas.iloc[Pay1_index,0])
        condition2 = p1_range_localmaximas["datetime"].le(p1_range_localminimas.iloc[Pay3_index,0])
        condition = condition1 & condition2
        
        #to find the y value of the 1st local maxima:
        Pay2 = p1_range_localmaximas[condition == True]['high'].values
        if len(Pay2) == 0:
            #proceed with the next pay3 index and rerun the while loop
            Pay3_index_range = Pay3_index_range + 1
            Pay3 = p1_range_localminimas.iloc[Pay3_index_range:,3].min()
            Pay3_index = compute_index_Pa_n(Pay3)
            continue
        else:
            Pay2=Pay2.max()
            break
    except IndexError as err1:
        break

1.09249


In [62]:
#Making a new dataframe of the higher highs and higher lows approaching P3 in chronological order
HH_HL_df = pd.DataFrame(columns=['datetime','price'])
HH_HL_df = pd.DataFrame({'datetime': pd.Series(dtype='datetime64[ns]'), 'price': pd.Series(dtype='float')})

#Grabbing the data attached to Pay*
Pa1 = p1_range_localminimas[p1_range_localminimas['low'] == Pay1].drop(['open','high','close'], axis=1)
Pa1 = Pa1.drop_duplicates(subset=['low'], keep='last')
Pa1 = Pa1.rename(columns={"low":"price"})
HH_HL_df = pd.concat([HH_HL_df, Pa1], ignore_index=True)

Pa2 = p1_range_localmaximas[p1_range_localmaximas['high'] == Pay2].drop(['open','low','close'], axis=1)
Pa2 = Pa2.drop_duplicates(subset=['high'], keep='last')
Pa2 = Pa2.rename(columns={"high":"price"})
HH_HL_df = pd.concat([HH_HL_df, Pa2], ignore_index=True)

Pa3 = p1_range_localminimas[p1_range_localminimas['low'] == Pay3].drop(['open','high','close'], axis=1)
Pa3 = Pa3.drop_duplicates(subset=['low'], keep='last')
Pa3 = Pa3.rename(columns={"low":"price"})
HH_HL_df = pd.concat([HH_HL_df, Pa3], ignore_index=True)


In [64]:
#Resetting the values to avoid issues
Pay = Pay2 = Pay1_index = Pay3_index = Pa1 = Pa2 = 0

#Assigning new values to find new sets of Pa2, Pa3
Pay1 = Pay3
Pay1_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay1].index[-1]

#Resetting the values to avoid issues
Pay3 = Pay3_index = 0

In [66]:
#Finding the initial value of Pay3
Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
Pay3_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay3].index[-1]


#Finding the values of the higher lows and higher highs and appending it to the HH_HL_df
while True:
    try:
        # Check if the current Pay3_index is beyond the end of p1_range_localminimas
        if Pay3_index > len(p1_range_localminimas):
            print(f"End of p1_range_localminimas reached.{Pay3_index}")
            break
            
        #finding the index of the next local maxima
        condition1 = p1_range_localmaximas["datetime"].gt(p1_range_localminimas.iloc[Pay1_index,0])
        condition2 = p1_range_localmaximas["datetime"].le(p1_range_localminimas.iloc[Pay3_index,0])
        condition = condition1 & condition2

        
        #to find the y value of the next local maxima:
        Pay2 = p1_range_localmaximas[condition == True]['high'].values
        
        #previous HH index
        pHH_index = HH_HL_df[HH_HL_df['price'] == Pay1].index[0] - 1
        pHHy = HH_HL_df.iloc[pHH_index,1]

        #if Pay2 isnt existing for the current Pay3
        condition3 = len(Pay2) == 0
        
        #if the current Pay2 is less than the previous HH
        try:
            condition4 = p1_range_localmaximas[condition == True]['high'].values.max() < pHHy
        except ValueError as err2:
            Pay3_index = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index:,3].min()
            Pay3_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay3].index[-1]
            continue
        
        if condition3 or condition4:
            #proceed with the next pay3 index and rerun the while loop
            Pay2 = 0
            Pay3_index_range = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index_range:,3].min()
            continue
        else:
            Pay2=Pay2.max()
            #Appending to the HH_HL_df 
            condition5 = p1_range_localmaximas['high'] == Pay2
            Pa2 = p1_range_localmaximas[condition5 & condition ].drop(['open','low','close'], axis=1)
            Pa2 = Pa2.drop_duplicates(subset=['high'], keep='last')
            Pa2 = Pa2.rename(columns={"high":"price"})
            HH_HL_df = pd.concat([HH_HL_df, Pa2], ignore_index=True)
    
            Pa3 = p1_range_localminimas[p1_range_localminimas['low'] == Pay3].drop(['open','high','close'], axis=1)
            Pa3 = Pa3.drop_duplicates(subset=['low'], keep='last')
            Pa3 = Pa3.rename(columns={"low":"price"})
            HH_HL_df = pd.concat([HH_HL_df, Pa3], ignore_index=True)
            
            #Assigning new values to find new sets of Pa2, Pa3
            Pay1 = Pay3
            Pay1_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay1].index[-1]
            
            #Finding the initial value of Pay3
            Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
            Pay3_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay3].index[-1]
            continue
    except IndexError as err1:
        break

# Values for Testing Purposes

In [70]:
#previous HH index
HH_HL_df.head(10)

,datetime,price
0,2024-09-08 09:27:00,1.09224
1,2024-09-08 09:37:00,1.09272
2,2024-09-08 09:51:00,1.09249
3,2024-09-08 09:58:00,1.09280
4,2024-09-08 10:01:00,1.09254
5,2024-09-08 10:03:00,1.09280
6,2024-09-08 10:06:00,1.09257
7,2024-09-08 10:08:00,1.09286
8,2024-09-08 10:09:00,1.09269


In [31]:
p1_range_localminimas

,datetime,open,high,low,close
0,2024-09-08 10:01:00,1.09261,1.09273,1.09254,1.09265
1,2024-09-08 10:06:00,1.09271,1.09271,1.09257,1.09270
2,2024-09-08 10:09:00,1.09279,1.09279,1.09269,1.09277


In [33]:
p1_range_localmaximas

,datetime,open,high,low,close
0,2024-09-08 10:03:00,1.09266,1.09280,1.09266,1.09276
1,2024-09-08 10:08:00,1.09280,1.09286,1.09276,1.09279
